# Навигация и краткое описание

**Разделы:**
- [LinearRegression](###```LinearRegression()```) 
- [Ridge](###```Ridge()```) 
- [RidgeCV](###```RidgeCV()```) 
- [SGDRegressor](###```SGDRegressor()```)
- [OrthogonalMatchingPursuit](###`OrthogonalMatchingPursuit()`)
- [ARDRegression()](###`ARDRegression()`)
- [PolynomialFeatures + LinearRegressor](###```PolynomialFeatures()```+```LinearRegressor()```)

# Подготовка к работе с моделями

In [4]:
import pandas as pd 

import sys
sys.path.append('..')
from utils import load_processed_data, ModelsRegressionHistory

In [5]:
# Заранее установим несколько констант
RANDOM_STATE = 42

# Загрузка данных
X_train, X_test, y_train, y_test = load_processed_data()
history_models = ModelsRegressionHistory()

# Создание моделей

## Линейные модели

In [6]:
from sklearn import linear_model

### ```LinearRegression()```

#### Описание метода

Линейная регрессия - модель, которая строит линейную зависимость между данными и предсказывает по этим зависиимостям целевой признак(*target*) в зависимости от числового значения признаков. Если разбирать линейную модель построенную для случая с двумя признаками, то предсказания будут строиться по формуле:
$$ 
    y = w_0 + w_1 \cdot x_1 + w_2 \cdot x_2 
$$
где:
- $y$ - целевая переменная
- $w_0$ - смещение
- $w_1$, $w_2$ - коэффициенты при признаках $x_1$ и $x_2$


На рисунке ниже представлена геометрическая интерпретация.

![Геометрический смысл линейной регрессии](../data/images/linearRegression/geometric_meaning.png)

Функцией потерь для линейной модели в ```sklearn``` это *SSE*, которая на практике часто используется как синоним MSE т.к. $MSE = \frac{1}{n}SSE$

Формула для рассчитывания потери
$$
Loss(SSE) = \sum^n_{i=2}(y_i-\overline{y}_i)^2
$$
где:
- y - истинное значение целевого переменной
- $\overline{y}$ - предсказанное значение переменной
- n - количество имеющихся обьектов

Теперь перейдём к тому, что отличает LinearRegression от остальных линейных регрессий из библиотеки ```sklearn```

Коэффициенты вычисляются аналитическим решением, которое вычисляет оптимальные коэффициенты за счёт вычисления псевдообратной матрицы. Псевдообратная матрица вычисляется в свою очередь через *SVD*(сингулярное разложение), что позволяет нам решить проблемы в случаях мультиколлинеарности и вырожденности данных.

Формула SVD([подробнее про SVD](http://machinelearning.ru/wiki/index.php?title=SVD)):
$$
X = VDU^T
$$
где: 

- V и U - ортогональные матрицы
- D - диагональная матрица сингулярных значений

Формула псевдообратной матрицы:
$$
X^+=VD^+U^T
$$
где:
- $D^+$ - диагональная матрица, где каждое сингулярное значение $σ_i$ заменяется на 1/$σ_i$, если $σ_i$ > 0, и на 0 иначе
- V и $U^T$ ортогональные матрицы, как и в SVG

После нахождения псевдообратной матрицы коэффициенты линейной регрессии рассчитываются по принципу перемножения псевдообратной матрицы $X^+$ на вектор $y$.
$$
w = X^+y
$$

#### Код и реализация

In [7]:
LR_standart = linear_model.LinearRegression()
LR_standart.fit(X_train, y_train)
y_pred = LR_standart.predict(X_test)

history_models.add_model(LR_standart, "linear_model", LR_standart.get_params(), "Линейная регрессия, с аналитическим оптимизатором", y_true=y_test, y_pred=y_pred)
history_models.to_dataframe()

,Модели,Класс модели,Параметры модели,Заметки,MAE,MSE,RMSE,R^2,MAPE,MSLE,RMSLE
0,LinearRegression,linear_model,"{'copy_X': True, 'fit_intercept': True, 'n_job...","Линейная регрессия, с аналитическим оптимизатором",0.2531,0.1038,0.3222,0.916,0.0278,0.001,0.0323


### ```Ridge()```

Класс ```Ridge()``` отличается от класса ```LinearRegression``` наличием L2-регуляризации. Мы можем контролировать силу регуляризации с помощью параметра ```alpha```(стандартное значение 1.0). 

Давайте рассмотрим, что такое L2-регуляризация для понимания сути различия ```Ridge()``` с ```LinearRegression```

Зачем нужна регуляризация?
* Уменьшает переобучение
* Решает проблему колинеарности
* Задача оптимизации становится устойчивее

L2-регуляризация([гребневая регуляризация](https://deepmachinelearning.ru/docs/Machine-learning/Base-concepts/Regularization)) добавляется к оптимизационнной функции модели штрафную функцию вида:
$$ 
alpha*R(w) = alpha*∑^{D}_{d=1}w^2_d 
$$
где:
- alpha - гиперпараметр задаваемый нами
- D - количество признаков
- w - веса модели 

Также отметим, что в классе ```Ridge``` мы можем изменить метод решения. Задав значение для переменной ```solver:{"auto", "svd", "cholesky", "lsqr", "sparse_cg", "sag", "saga", "lbfgs"}, default="auto"```. Рассматривать подробно каждый вариант функции оптимизации мы не будем, так как ```Ridge()``` может автоматически подобрать наиболее походящий алгоритм в зависимости от данных.

In [8]:
LR_Ridge = linear_model.Ridge()
LR_Ridge.fit(X_train, y_train)
y_pred = LR_Ridge.predict(X_test)

history_models.add_model(LR_Ridge, "linear_model", LR_Ridge.get_params(), "Линейная регрессия, с регуляризацией", y_true=y_test, y_pred=y_pred)
history_models.to_dataframe()

,Модели,Класс модели,Параметры модели,Заметки,MAE,MSE,RMSE,R^2,MAPE,MSLE,RMSLE
0,LinearRegression,linear_model,"{'copy_X': True, 'fit_intercept': True, 'n_job...","Линейная регрессия, с аналитическим оптимизатором",0.2531,0.1038,0.3222,0.916,0.0278,0.001,0.0323
1,Ridge,linear_model,"{'alpha': 1.0, 'copy_X': True, 'fit_intercept'...","Линейная регрессия, с регуляризацией",0.2531,0.1038,0.3222,0.916,0.0278,0.001,0.0323


### ```RidgeCV()```

Рассмотрим ```RidgeCV()```. В отличие от класса ```Ridge()```, он  наличием кросс-валидации параметра ```alpha```, который отвечает за величину штрафа за большое значение коэффициента.

In [9]:
LR_RidgeCV = linear_model.RidgeCV(alphas=[0.2, 0.6, 1.0, 1.5, 3.8, 10.0, 14.0])
LR_RidgeCV.fit(X_train, y_train)
y_pred = LR_RidgeCV.predict(X_test)


In [10]:
history_models.add_model(LR_RidgeCV, "linear_model", LR_RidgeCV.get_params(), "Линейная регрессия, с регуляризацией и кросс-валидацией", y_true=y_test, y_pred=y_pred)
history_models.to_dataframe()

,Модели,Класс модели,Параметры модели,Заметки,MAE,MSE,RMSE,R^2,MAPE,MSLE,RMSLE
0,LinearRegression,linear_model,"{'copy_X': True, 'fit_intercept': True, 'n_job...","Линейная регрессия, с аналитическим оптимизатором",0.2531,0.1038,0.3222,0.916,0.0278,0.001,0.0323
1,Ridge,linear_model,"{'alpha': 1.0, 'copy_X': True, 'fit_intercept'...","Линейная регрессия, с регуляризацией",0.2531,0.1038,0.3222,0.916,0.0278,0.001,0.0323
2,RidgeCV,linear_model,"{'alpha_per_target': False, 'alphas': [0.2, 0....","Линейная регрессия, с регуляризацией и кросс-в...",0.2531,0.1038,0.3222,0.916,0.0278,0.001,0.0323


### ```SGDRegressor()```

Давайте разберём по пунктам класс `SGDRegressor()`.
- Наша модель изменяется с помощью *SGD(стохастический [градиентный спуск](https://neurohive.io/ru/osnovy-data-science/gradient-descent/))* это один из ключевых методов в машинном обучении, особенно эффективно при работе с большими объёмами данных.
- С помощью параметра `penalty` можем назначить один из способов регуляризации `['l2', 'l1', 'elasticnet']`.
- Параметр `loss` определяеи тип функцию потерь, которую будет минимизировать модель `["squared_error", "huber", "epsilon_insensitive", "squared_epsilon_insensitive"]`:
    - `"squared_error"` — стандартная среднеквадратичная ошибка (по умолчанию),
    - `"huber"` — более устойчива к выбросам,
    - `"epsilon_insensitive"` и `"squared_epsilon_insensitive"` — используются в задачах поддержки вектора (аналогично `SVR`).

- С помощью параметра ```learning_rate``` можно изменять скорость обучения:
    - `"constant"` — постоянная скорость обучения,
    - `"optima"` — автоматически выбирает оптимальное значение,
    - `"invscaling"` - скорость уменьшается со временем по определённой формуле,
    - `"adaptive"` - скорость уменьшается только при отсутствии улучшения.

- Параметр `"max_iter"` определяет максимальное количество итераций(эпох) обучения. При работе с большим количеством данных важно установить это значение разумно.

- С помощью задания `"early_stopping=True"` можно включить механизм остановки обучения при прекращении улучшения функции потерь.


In [11]:
LR_SGDRegressor = linear_model.SGDRegressor()
LR_SGDRegressor.fit(X_train, y_train)
y_pred = LR_SGDRegressor.predict(X_test)

c:\Users\sinde\OneDrive\Рабочий стол\classic_ml_skills\.venv\Lib\site-packages\sklearn\utils\validation.py:1406: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [12]:
history_models.add_model(LR_SGDRegressor, "linear_model", LR_SGDRegressor.get_params(), "Линейная регрессия, с SGD", y_true=y_test, y_pred=y_pred)
history_models.to_dataframe()

,Модели,Класс модели,Параметры модели,Заметки,MAE,MSE,RMSE,R^2,MAPE,MSLE,RMSLE
0,LinearRegression,linear_model,"{'copy_X': True, 'fit_intercept': True, 'n_job...","Линейная регрессия, с аналитическим оптимизатором",0.2531,0.1038,0.3222,0.9160,0.0278,0.0010,0.0323
1,Ridge,linear_model,"{'alpha': 1.0, 'copy_X': True, 'fit_intercept'...","Линейная регрессия, с регуляризацией",0.2531,0.1038,0.3222,0.9160,0.0278,0.0010,0.0323
2,RidgeCV,linear_model,"{'alpha_per_target': False, 'alphas': [0.2, 0....","Линейная регрессия, с регуляризацией и кросс-в...",0.2531,0.1038,0.3222,0.9160,0.0278,0.0010,0.0323
3,SGDRegressor,linear_model,"{'alpha': 0.0001, 'average': False, 'early_sto...","Линейная регрессия, с SGD",0.2536,0.1045,0.3233,0.9154,0.0279,0.0011,0.0324


### `OrthogonalMatchingPursuit()`

Отличие данного класса от остальных заключается в его способности уменьшать количество признаков. Он находит самые влиятельные признаки из имеющихся и на их основе строит модель. (*жадно отбирает признаки*)

Алгоритм отбора признаков:
1) Инициализация:
    - Создаётся пустой список признаков.
    - Остаток r = y (это то, что модель ещё не обьяснила)
2) На каждом шаге:
    - Вычисляется корреляция между каждым признаком и текущим остстатком r
    - Признак с максимальной корреляцией добавляем в список выбранных
    - Строим модель по методу наименьших квадратов(используем только выбранные признаки)
    - обновляем остаток $r = y - X_{выбранные}*w$
3) Повторяем 2 пукт пока не удоврится одно из следующих вариантов:
    - не выбрано заданное количество признаков в гиперпараметре `n_nonzero_coefs`
    - r минимален
    - не закончились признаки 

In [13]:
LR_OrthogonalMatchingPursuit = linear_model.OrthogonalMatchingPursuit(
    # у нас есть больше 70 признков, давайте попробуем задать максисальное количество признаков равное 50
    n_nonzero_coefs=50
)
LR_OrthogonalMatchingPursuit.fit(X_train, y_train)
y_pred = LR_OrthogonalMatchingPursuit.predict(X_test)

In [14]:
history_models.add_model(LR_OrthogonalMatchingPursuit, "linear_model", LR_OrthogonalMatchingPursuit.get_params(), "Линейная регрессия, c использованием OMP", y_true=y_test, y_pred=y_pred)
print('Параметры:', LR_OrthogonalMatchingPursuit.get_params())
history_models.to_dataframe()

Параметры: {'fit_intercept': True, 'n_nonzero_coefs': 50, 'precompute': 'auto', 'tol': None}


,Модели,Класс модели,Параметры модели,Заметки,MAE,MSE,RMSE,R^2,MAPE,MSLE,RMSLE
0,LinearRegression,linear_model,"{'copy_X': True, 'fit_intercept': True, 'n_job...","Линейная регрессия, с аналитическим оптимизатором",0.2531,0.1038,0.3222,0.9160,0.0278,0.0010,0.0323
1,Ridge,linear_model,"{'alpha': 1.0, 'copy_X': True, 'fit_intercept'...","Линейная регрессия, с регуляризацией",0.2531,0.1038,0.3222,0.9160,0.0278,0.0010,0.0323
2,RidgeCV,linear_model,"{'alpha_per_target': False, 'alphas': [0.2, 0....","Линейная регрессия, с регуляризацией и кросс-в...",0.2531,0.1038,0.3222,0.9160,0.0278,0.0010,0.0323
3,SGDRegressor,linear_model,"{'alpha': 0.0001, 'average': False, 'early_sto...","Линейная регрессия, с SGD",0.2536,0.1045,0.3233,0.9154,0.0279,0.0011,0.0324
4,OrthogonalMatchingPursuit,linear_model,"{'fit_intercept': True, 'n_nonzero_coefs': 50,...","Линейная регрессия, c использованием OMP",0.2535,0.1041,0.3226,0.9157,0.0279,0.0010,0.0323


### `ARDRegression()`

Байевский метод линейной регрессии, который автоматически определяет релевантные признаки. Он использует отдельную регуляризацию для каждого признака, постепенно обнуляя коэфициенты неинформативных признаков.

In [15]:
LR_ARDRegression = linear_model.ARDRegression()
LR_ARDRegression.fit(X_train, y_train)
y_pred = LR_ARDRegression.predict(X_test)

c:\Users\sinde\OneDrive\Рабочий стол\classic_ml_skills\.venv\Lib\site-packages\sklearn\utils\validation.py:1406: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [16]:
history_models.add_model(LR_ARDRegression, "linear_model", LR_ARDRegression.get_params(), "Линейная регрессия, c байевского метода", y_true=y_test, y_pred=y_pred)
history_models.to_dataframe()

,Модели,Класс модели,Параметры модели,Заметки,MAE,MSE,RMSE,R^2,MAPE,MSLE,RMSLE
0,LinearRegression,linear_model,"{'copy_X': True, 'fit_intercept': True, 'n_job...","Линейная регрессия, с аналитическим оптимизатором",0.2531,0.1038,0.3222,0.9160,0.0278,0.0010,0.0323
1,Ridge,linear_model,"{'alpha': 1.0, 'copy_X': True, 'fit_intercept'...","Линейная регрессия, с регуляризацией",0.2531,0.1038,0.3222,0.9160,0.0278,0.0010,0.0323
2,RidgeCV,linear_model,"{'alpha_per_target': False, 'alphas': [0.2, 0....","Линейная регрессия, с регуляризацией и кросс-в...",0.2531,0.1038,0.3222,0.9160,0.0278,0.0010,0.0323
3,SGDRegressor,linear_model,"{'alpha': 0.0001, 'average': False, 'early_sto...","Линейная регрессия, с SGD",0.2536,0.1045,0.3233,0.9154,0.0279,0.0011,0.0324
4,OrthogonalMatchingPursuit,linear_model,"{'fit_intercept': True, 'n_nonzero_coefs': 50,...","Линейная регрессия, c использованием OMP",0.2535,0.1041,0.3226,0.9157,0.0279,0.0010,0.0323
5,ARDRegression,linear_model,"{'alpha_1': 1e-06, 'alpha_2': 1e-06, 'compute_...","Линейная регрессия, c байевского метода",0.2531,0.1038,0.3222,0.9160,0.0278,0.0010,0.0323


### ```PolynomialFeatures()``` + ```LinearRegressor()```

Полиномиальная регрессия - это метод машинного обучения, который используется в моделировании нелинейных зависимостей между переменными. Этот метод позволит нам найти более сложные зависимости между имеющимися атрибутами.

Обозначим преимущества и недостатки этого метода

**Преимущества**
* Гибкость - может найти сложные зависимости
* Подходит для анализа данных с криволинейными трендами (к примеру: валюта начала скакать, то вверх, то вниз в зависимости от месяца года а наши авиакомпания при формировании цены ссылаются на курс валюты)

**Недостатки**
* Возможно переобучение
* Чувствительна к выбросам

In [17]:
X_train

,duration,days_left,stops_encoded,flight_value,airline_Air_India,airline_GO_FIRST,airline_Indigo,airline_SpiceJet,airline_Vistara,class_Economy,...,time_interval_Morning_Evening,time_interval_Morning_Late_Night,time_interval_Morning_Morning,time_interval_Morning_Night,time_interval_Night_Afternoon,time_interval_Night_Early_Morning,time_interval_Night_Evening,time_interval_Night_Late_Night,time_interval_Night_Morning,time_interval_Night_Night
0,0.377227,0.312500,1,0.910313,0.0,0.0,0.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,0.430063,0.687500,1,0.925480,0.0,0.0,0.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,0.334426,0.375000,1,0.931547,0.0,0.0,0.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.015359,0.895833,0,0.922649,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,0.170592,0.583333,1,0.921234,0.0,0.0,0.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
240013,0.354905,0.854167,1,0.945804,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
240014,0.143354,0.791667,1,0.912841,0.0,0.0,0.0,0.0,1.0,1.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
240015,0.491501,0.416667,1,0.931547,0.0,0.0,0.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
240016,0.438665,0.520833,1,0.938625,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [18]:
from sklearn.preprocessing import PolynomialFeatures

poly = PolynomialFeatures(degree=2)
X_poly_train = poly.fit_transform(X_train)
X_poly_test = poly.transform(X_test)

Выше мы добавили полиномиальные признаки во второй степени.

Пример:
 1) У нас есть признаки *a* и *b*. 
 2) Обрабатываем с помощью класса ```PolynomialFeatures``` 
 3) На выходе мы получаем атрибуты вида *$1, a, b, a^2, b^2, ab$*

Мы имеем 83 признака.
 
Если мы будем далее использовать все имеющиеся признаки то мы получим 
$$\frac{83*84}{2} + 1 = 3486$$

Это займёт очень много оперативной памяти при обучении линейной регресси. Поэтому мы переведём полученную матрицу в разряженную матрицу с помощью метода из библиотеки `scipy`

In [19]:
from scipy.sparse import csr_matrix

X_poly_train = csr_matrix(X_poly_train)
X_poly_test = csr_matrix(X_poly_test)

Теперь когда мы преобразовали наши признаки к полиномиальному виду в разряженной матрице мы можем использовать любую из разобранных нами линейных регрессий. Возьмём стандартную линейную регрессию.

In [20]:
PLR_standart = linear_model.LinearRegression()
PLR_standart.fit(X_poly_train, y_train)
y_pred = PLR_standart.predict(X_poly_test)

In [21]:
history_models.add_model(PLR_standart, "poly_linear_model", PLR_standart.get_params(), "Полиномиальная линейная регрессия", y_true=y_test, y_pred=y_pred)

## Итоги

In [22]:
history_models.to_dataframe()

,Модели,Класс модели,Параметры модели,Заметки,MAE,MSE,RMSE,R^2,MAPE,MSLE,RMSLE
0,LinearRegression,linear_model,"{'copy_X': True, 'fit_intercept': True, 'n_job...","Линейная регрессия, с аналитическим оптимизатором",0.2531,0.1038,0.3222,0.9160,0.0278,0.0010,0.0323
1,Ridge,linear_model,"{'alpha': 1.0, 'copy_X': True, 'fit_intercept'...","Линейная регрессия, с регуляризацией",0.2531,0.1038,0.3222,0.9160,0.0278,0.0010,0.0323
2,RidgeCV,linear_model,"{'alpha_per_target': False, 'alphas': [0.2, 0....","Линейная регрессия, с регуляризацией и кросс-в...",0.2531,0.1038,0.3222,0.9160,0.0278,0.0010,0.0323
3,SGDRegressor,linear_model,"{'alpha': 0.0001, 'average': False, 'early_sto...","Линейная регрессия, с SGD",0.2536,0.1045,0.3233,0.9154,0.0279,0.0011,0.0324
4,OrthogonalMatchingPursuit,linear_model,"{'fit_intercept': True, 'n_nonzero_coefs': 50,...","Линейная регрессия, c использованием OMP",0.2535,0.1041,0.3226,0.9157,0.0279,0.0010,0.0323
5,ARDRegression,linear_model,"{'alpha_1': 1e-06, 'alpha_2': 1e-06, 'compute_...","Линейная регрессия, c байевского метода",0.2531,0.1038,0.3222,0.9160,0.0278,0.0010,0.0323
6,LinearRegression,poly_linear_model,"{'copy_X': True, 'fit_intercept': True, 'n_job...",Полиномиальная линейная регрессия,0.1804,0.0579,0.2407,0.9531,0.0199,0.0006,0.0242


Подведём итог работы с линейными методами:
* линейные модели показывают не плохие результаты;
* значительное изменение метрик в лучшую сторону показала полиномиальная модель, что явно говорит нам о том что наши данные нелинейные;

Далее мы можем поэксперементировать подставляя разные линейные модели для наших полиномиальных параметров, но предлагаю пропустить эти дейтсвия момент и перейти к более сложным методам, а именно к деревьям;

In [23]:
history_models.to_new_csv("linear_models.csv")